In [1]:
# Library
import pandas as pd
import numpy as np
from pathlib import Path
import os
from typing import Union, List, Tuple
import random

In [4]:
# Files manager
cwd = Path(os.getcwd())
os.listdir(cwd)

['listing_0522.xlsx', '.ipynb_checkpoints', 'anonymise.ipynb', 'README.rst']

In [5]:
def anonymise_cat(
    df:pd.DataFrame, 
    col_name: str, 
    alt: Union[pd.DataFrame, str],
)-> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Anonymise a categorical column
    
    Parameters
    ----------
        df: pd.DataFrame
            Input DataFrame with the column to anonymise
            
        col_name: str
            Name of the column to anonymise
            
        alt: str or pd.DataFrame
            if str, the name of alternative data to create. For example, 
            if alt = "CLIENT", data will be replaced with
            CLIENT_1, CLIENT_2, etc ....
            
            if pd.DataFrame, the daatframe must be the merge key with another field. 
            For example, if two columns related to customers are meant to be merged between 
            table A and B, then after the anonymisation of customer field in table A, one gets
            a transcoding table to use as an alt parameter for table B
            
    Returns
    -------
        df: pd.DataFrame
            The previous dataframe with the column col_name anonymised
            
        df_to_merge: pd.DataFrame
            The transcoding table between col_name's elements and the new anonymised words      
    """
    if isinstance(alt, str):
        # If alt value is a string --> create a transcoding table
        # -------------------------------------------------------
        col_values = df[col_name].value_counts()
        alternative_values = [f"{alt} {i}" for i in range(1,col_values.shape[0] + 1)]
        df_to_merge = pd.DataFrame()
        df_to_merge["before"] = list(col_values.index)
        df_to_merge["after"] = alternative_values
        
    else:
        # If alt value is a dataframe --> update the current transcoding table
        # --------------------------------------------------------------------
        col_values = list(df[col_name].value_counts().index)
        new_before = pd.DataFrame()
        new_before["before"] = col_values
        df_to_merge = pd.merge(
            alt,
            new_before,
            on = "before",
            how = "outer"
        )
        # get alt str info : Name and Max now
        df_to_merge["test_name"] = df_to_merge["after"].str.split().str[0].astype(str)
        df_to_merge["test_value"] = df_to_merge["after"].str.split().str[1].astype(str)
        last_value = df_to_merge.loc[
            df_to_merge.test_value.str.isdigit(),
            "test_value"
        ].astype(int).max()
        name = str(df_to_merge.loc[0,"test_name"]).strip()
        nb_row_to_add = df_to_merge[df_to_merge.after.isna()].shape[0]
        list_to_add = [f"{name} {i}" for i in range(last_value + 1, last_value + 1 + nb_row_to_add)]
        df_to_merge.loc[df_to_merge.after.isna(), "after"] = list_to_add
        df_to_merge = df_to_merge[["before","after"]]
             
    # Update the columns data according to the transcoding table
    # ----------------------------------------------------------
    df = pd.merge(
        df,
        df_to_merge,
        left_on = col_name,
        right_on = "before",
        how = "left",
        validate = "m:1"
    )
    df.drop([col_name,"before"], axis=1, inplace=True)
    df.rename(
        columns = {
            "after":col_name,
        },
        inplace=True
    )
    
    return(df, df_to_merge)

In [6]:
def anonymise_float(
    df:pd.DataFrame,
    col_name: str,
    range_values: Union[List,float],
)-> pd.DataFrame:
    """
    Anonymise a numerical column
    
    Parameters
    ----------
        df: pd.DataFrame
            Input DataFrame with the column to anonymise
            
        col_name: str
            Name of the column to anonymise
            
        range_values: List or float
            If float, the factor that will be multiplied with df column's fields
            
            If List, the interval in which a random float is chosen as the 
            multiplication factor
            
    Returns
    -------
        df: pd.DataFrame
            The previous dataframe with the column col_name anonymised         
    """
    # Transform the column into a fully readable one by python
    # --------------------------------------------------------
    df[col_name] = df[col_name].astype(str).str.replace(',','.').astype(float)

    if isinstance(range_values, float):
        # if float --> update df with a simple multiplication
        # ---------------------------------------------------
        df[col_name] = df[col_name] * range_values
        
    else:
        # If range --> find uniformaly a factor in the interval and then multiply it
        # --------------------------------------------------------------------------
        min_range, max_range = range_values[0], range_values[1]
        coeff = random.uniform(min_range, max_range)
        df[col_name] = df[col_name] * coeff
        
    # Transform the column into a fully readable one by excel
    # -------------------------------------------------------
    df[col_name] = df[col_name].astype(str).str.replace('.',',')
    
    return (df)

In [7]:
df = pd.read_excel(Path(cwd,'listing_0522.xlsx'))

In [10]:
df.columns

Index(['Mon niveau d'expérience actuel', 'Sujets par ordre de préférence',
       'Mon niveau d'etude', 'Disponibilité pour le 31 Mai',
       'Moyen de transport', 'Cybersécurité', 'Data / IA', 'Cloud',
       'Développement', 'Mainframe', 'ID'],
      dtype='object')

In [11]:
# Niveau d'expérience
df,_ = anonymise_cat(df, "Mon niveau d'expérience actuel", "EXPERIENCE")

In [13]:
df

,Sujets par ordre de préférence,Mon niveau d'etude,Disponibilité pour le 31 Mai,Moyen de transport,Cybersécurité,Data / IA,Cloud,Développement,Mainframe,ID,Mon niveau d'expérience actuel
0,"1 Développement , 2 Cloud , 3 Data / IA , 4 M...",Bac + 5/M2 et +,13h30 - 19h,Je souhaite prendre la navette depuis Porte Ma...,30 pts,50 pts,40 pts,50 pts,20 pts,0522-01,EXPERIENCE 1
1,"1 Cloud , 2 Mainframe , 3 Cybersécurité , 4 D...",Bac + 5/M2 et +,13h30 - 19h,Je souhaite prendre la navette depuis Porte Ma...,30 pts,20 pts,70 pts,10 pts,60 pts,0522-02,EXPERIENCE 1
2,"1 Mainframe , 2 Développement , 3 Data / IA ,...",Bac + 3 / L3,Tout me va,Je souhaite venir par mes propres moyens,20 pts,30 pts,10 pts,60 pts,70 pts,0522-03,EXPERIENCE 1
3,"1 Mainframe , 2 Développement , 3 Data / IA ,...",Bac + 5/M2 et +,13h30 - 19h,Je souhaite prendre la navette depuis Orsay,20 pts,30 pts,10 pts,40 pts,50 pts,0522-04,EXPERIENCE 2
4,"1 Cybersécurité , 2 Cloud , 3 Mainframe , 4 D...",Bac + 5/M2 et +,Tout me va,Je souhaite prendre la navette depuis Porte Ma...,50 pts,20 pts,60 pts,30 pts,50 pts,0522-05,EXPERIENCE 1
5,"1 Mainframe , 2 Développement , 3 Data / IA ,...",Bac + 5/M2 et +,13h30 - 19h,Je souhaite venir par mes propres moyens,20 pts,30 pts,10 pts,60 pts,70 pts,0522-06,EXPERIENCE 1
6,"1 Data / IA , 2 Cloud , 3 Cybersécurité , 4 M...",Bac + 4/M1,13h30 - 19h,Je souhaite venir par mes propres moyens,30 pts,70 pts,60 pts,10 pts,40 pts,0522-07,EXPERIENCE 1
7,"1 Développement , 2 Data / IA , 3 Mainframe ,...",Bac + 5/M2 et +,13h30 - 19h,Je souhaite venir par mes propres moyens,40 pts,40 pts,10 pts,70 pts,50 pts,0522-08,EXPERIENCE 1
8,"1 Mainframe , 2 Développement , 3 Data / IA ,...",Bac + 5/M2 et +,Tout me va,Je souhaite venir par mes propres moyens,20 pts,50 pts,10 pts,40 pts,70 pts,0522-09,EXPERIENCE 2
9,"1 Cloud ,2 Cybersécurité ,3 Data / IA ,4 Mainf...",Bac + 5/M2 et +,13h30 - 19h,Je souhaite venir par mes propres moyens,40 pts,30 pts,50 pts,10 pts,20 pts,0522-10,EXPERIENCE 1


In [14]:
_

,before,after
0,0-2 ans,EXPERIENCE 1
1,3-5 ans,EXPERIENCE 2


#### Anonym_Ristournes_N-1.csv

In [4]:
df1 = pd.read_csv(Path(cwd,'TO_DO/Anonym_Ristournes_N-1.csv'), sep = ";")

In [5]:
df1.head()

,CENTRALE,CODE_SOCIETE,LIBELLE_SOCIETE,CODE_CLIENT_BCP,POLE_AFFILIANT,FOURNISSEUR,NOM_FOURNISSEUR,REGROUPEMENT,CA_DECLARE_FRN,CA_FRN_CRF,...,SOURCES,Carpr,Mteca,_Ca_Bn,_Ristourne_Prolog,Ecart_CA,%CA,Ecart_Ristourne,%Ristourne,DATE
0,CAP,144,C.S.F.,209,ENT GR,195,KAMBLY,1,"0,0","106697,2",...,NR,"106793,03","894,64","105898,39","37051,069554","-798,8099999999977","-0,007486700681929776","-279,5304460000043","-0,007487970887154353",31/05/2022
1,CAP,144,C.S.F.,209,ENT GR,195,KAMBLY,13,"0,0","20971,0",...,NR,"29909,74","254,51999999999998","29655,219999999998","624,303762","8684,219999999998","0,4141061465833769","-9,016238000000044","-0,014236464978210135",31/05/2022
2,CAP,144,C.S.F.,209,ENT GR,301,LOTUS BAKERIES FRANC,2,"0,0","165268,26",...,NR,"167239,03","-417,93","167656,96000000002","63731,108765000004","2388,7000000000116","0,01445347098105838","931,5287650000027","0,014833359793170634",31/05/2022
3,CAP,144,C.S.F.,209,ENT GR,301,LOTUS BAKERIES FRANC,9,"0,0","220388,52000000002",...,NR,"216891,25999999998","0,0","216891,25999999998","80418,847853","-3497,2600000000384","-0,01586861239414847","-1317,7221470000077","-0,01612157381940553",31/05/2022
4,CAP,144,C.S.F.,209,ENT GR,592,REGILAIT,1,"0,0","151721,24",...,NR,"143007,51","77,35","142930,16","40113,0493295","-8791,079999999987","-0,05794231578914058","-2519,6106705000057","-0,059100480019309266",31/05/2022


In [6]:
df1.sample(1).T

,10000
CENTRALE,CAP
CODE_SOCIETE,144
LIBELLE_SOCIETE,C.S.F.
CODE_CLIENT_BCP,27269
POLE_AFFILIANT,ENT GR
FOURNISSEUR,4240
NOM_FOURNISSEUR,FLEURY MICHON
REGROUPEMENT,30
CA_DECLARE_FRN,"0,0"
CA_FRN_CRF,"221289,11"


In [7]:
columns_order = list(df1.columns)

# CENTRALE
df1,_ = anonymise_cat(df1, "CENTRALE", "CENT")
# CODE_SOCIETE
# LIBELLE_SOCIETE
df1,_ = anonymise_cat(df1, "LIBELLE_SOCIETE", "LIB")
# CODE CLIENT BCP
# POLE_AFFILIANT
df1,_ = anonymise_cat(df1, "POLE_AFFILIANT", "POLE")
# FOURNISSEUR
# NOM_FOURNISSEUR
df1,_ = anonymise_cat(df1, "NOM_FOURNISSEUR", "FRN")
# REGROUPEMENT
# CA_DECLARE_FRN
df1 = anonymise_float(df1, "CA_DECLARE_FRN", [0.1,1.2])
# CA_FRN_CRF
df1 = anonymise_float(df1, "CA_FRN_CRF", [0.1,1.2])
# TOTAL_CA_DECLARE_FRN
df1 = anonymise_float(df1, "TOTAL_CA_DECLARE_FRN", [0.1,1.2])
# TOTAL_ENCAISSE
df1 = anonymise_float(df1, "TOTAL_ENCAISSE", [0.1,1.2])
# MTT_A_REVERSER_DIFF
df1 = anonymise_float(df1, "MTT_A_REVERSER_DIFF", [0.1,1.2])
# MTT_A_REV_DIFF_HORS_SYN
df1 = anonymise_float(df1, "MTT_A_REV_DIFF_HORS_SYN", [0.1,1.2])
# MTT_A_REVERSER_BCP
df1 = anonymise_float(df1, "MTT_A_REVERSER_BCP", [0.1,1.2])
# ECART_MTT_A_REVERSER
df1 = anonymise_float(df1, "ECART_MTT_A_REVERSER", [0.1,1.2])
# MTT_PREVISION_AVEC_SYNERGIE
df1 = anonymise_float(df1, "MTT_PREVISION_AVEC_SYNERGIE", [0.1,1.2])
# MTT_PREVISION_HORS_SYNERGIE
df1 = anonymise_float(df1, "MTT_PREVISION_HORS_SYNERGIE", [0.1,1.2])
# Cdfop
# Norca
# Cient
# Cdprolog
# Lbetblba
df1, key_lbetblba = anonymise_cat(df1, "Lbetblba", "LBL")
# Activite
df1,_ = anonymise_cat(df1, "Activite", "ACT")
# Code_Ent_BCP
# Libelle_entrepot
# SOURCES
# Carpr
df1 = anonymise_float(df1,"Carpr",[0.1,1.2])
# Mteca
df1 = anonymise_float(df1,"Mteca",[0.1,1.2])
# _Ca_Bn
df1 = anonymise_float(df1,"_Ca_Bn",[0.1,1.2])
# _Ristourne_Prolog
df1 = anonymise_float(df1,"_Ristourne_Prolog",[0.1,1.2])
# Ecart_CA
df1 = anonymise_float(df1,"Ecart_CA",[0.1,1.2])
# %CA
df1 = anonymise_float(df1,"%CA",[0.1,1.2])
# Ecart_Ristourne
df1 = anonymise_float(df1,"Ecart_Ristourne",[0.1,1.2])
# %Ristourne
df1 = anonymise_float(df1,"%Ristourne",[0.1,1.2])
# DATE

df1 = df1[columns_order]

In [8]:
df1.head()

,CENTRALE,CODE_SOCIETE,LIBELLE_SOCIETE,CODE_CLIENT_BCP,POLE_AFFILIANT,FOURNISSEUR,NOM_FOURNISSEUR,REGROUPEMENT,CA_DECLARE_FRN,CA_FRN_CRF,...,SOURCES,Carpr,Mteca,_Ca_Bn,_Ristourne_Prolog,Ecart_CA,%CA,Ecart_Ristourne,%Ristourne,DATE
0,CENT 1,144,LIB 1,209,POLE 1,195,FRN 84,1,"0,0","122007,98859282114",...,NR,"116636,76134778754","370,09348374169423","56737,57841475099","37861,98668170696","-931,4141847532301","-0,003031187718950118","-283,7986752121471","-0,004008134074002164",31/05/2022
1,CENT 1,144,LIB 1,209,POLE 1,195,FRN 84,13,"0,0","23980,287475023266",...,NR,"32666,69375664662","105,28949463687741","15888,488674442471","637,9675676496546","10125,819270562106","0,1676617670711713","-9,153909480748982","-0,007620443673250927",31/05/2022
2,CENT 1,144,LIB 1,209,POLE 1,301,FRN 74,2,"0,0","188983,85319235557",...,NR,"182654,41883375237","-172,88872581168545","89826,19957469392","65125,95777441852","2785,229357569457","0,005851867944945447","945,7525404191721","0,007939947378926736",31/05/2022
3,CENT 1,144,LIB 1,209,POLE 1,301,FRN 74,9,"0,0","252013,73638810334",...,NR,"236883,3820993836","0,0","116204,64552600039","82178,93256578846","-4077,81271112045","-0,006424825173259632","-1337,8428180818005","-0,00862949794088436",31/05/2022
4,CENT 1,144,LIB 1,209,POLE 1,592,FRN 234,1,"0,0","173492,868783892",...,NR,"156189,3394616797","31,99804498727985","76578,22900643632","40990,982386154064","-10250,418261289215","-0,023459470798864755","-2558,0833163993884","-0,031635091979544255",31/05/2022


In [9]:
df1.sample(1).T

,326
CENTRALE,CENT 1
CODE_SOCIETE,144
LIBELLE_SOCIETE,LIB 1
CODE_CLIENT_BCP,209
POLE_AFFILIANT,POLE 1
FOURNISSEUR,95779
NOM_FOURNISSEUR,FRN 4
REGROUPEMENT,3
CA_DECLARE_FRN,"0,0"
CA_FRN_CRF,"3676386,6095723384"


In [10]:
# Save
path_output = Path(cwd,'DONE/Anonym_Ristournes_N-1_2605.csv')
df1.to_csv(path_output, sep = ";", encoding = 'utf-8', index = False)

#### Anonym_Ristournes_N.csv

In [11]:
df2 = pd.read_csv(Path(cwd,'TO_DO/Anonym_Ristournes_N.csv'), sep = ";")

In [12]:
df2.head()

,CENTRALE,CODE_SOCIETE,LIBELLE_SOCIETE,CODE_CLIENT_BCP,POLE_AFFILIANT,FOURNISSEUR,NOM_FOURNISSEUR,REGROUPEMENT,CA_DECLARE_FRN,CA_FRN_CRF,...,SOURCES,Carpr,Mteca,_Ca_Bn,_Ristourne_Prolog,Ecart_CA,%CA,Ecart_Ristourne,%Ristourne,DATE
0,CAP,144,C.S.F.,209,ENT GR,195,KAMBLY,1,0,"216131,08",...,NR,"215780,64","1164,56","214616,08","75210,97874",-1515,"-0,007009635","-530,911256","-0,00700948",30/09/2022
1,CAP,144,C.S.F.,209,ENT GR,195,KAMBLY,13,0,"60527,64",...,NR,"69780,99","356,78","69424,21","1825,32726","8896,57","0,146983593","-2,60274","-0,001423873",30/09/2022
2,CAP,144,C.S.F.,209,ENT GR,301,LOTUS BAKERIES FRANC,2,0,"327105,12",...,NR,"329128,09","-1509,77","330637,86","127277,3617","3532,74","0,010800014","1377,591675","0,010941971",30/09/2022
3,CAP,144,C.S.F.,209,ENT GR,301,LOTUS BAKERIES FRANC,9,0,"386969,59",...,NR,"386172,77","-656,14","386828,91","144451,3544","-140,68","-0,000363543","-52,965627","-0,000366533",30/09/2022
4,CAP,144,C.S.F.,209,ENT GR,592,REGILAIT,1,0,"265823,65",...,NR,"247583,11","77,35","247505,76","70126,24653","-18317,89","-0,068909933","-5253,81347","-0,069697656",30/09/2022


In [13]:
df2.sample(1).T

,3209
CENTRALE,CAP
CODE_SOCIETE,144
LIBELLE_SOCIETE,C.S.F.
CODE_CLIENT_BCP,8499
POLE_AFFILIANT,ENT GR
FOURNISSEUR,9297
NOM_FOURNISSEUR,FROMARSAC
REGROUPEMENT,1
CA_DECLARE_FRN,0
CA_FRN_CRF,"1559428,31"


In [14]:
columns_order = list(df2.columns)

# CENTRALE
df2,_ = anonymise_cat(df2, "CENTRALE", "CENT")
# CODE_SOCIETE
# LIBELLE_SOCIETE
df2,_ = anonymise_cat(df2, "LIBELLE_SOCIETE", "LIB")
# CODE CLIENT BCP
# POLE_AFFILIANT
df2,_ = anonymise_cat(df2, "POLE_AFFILIANT", "POLE")
# FOURNISSEUR
# NOM_FOURNISSEUR
df2,_ = anonymise_cat(df2, "NOM_FOURNISSEUR", "FRN")
# REGROUPEMENT
# CA_DECLARE_FRN
df2 = anonymise_float(df2, "CA_DECLARE_FRN", [0.1,1.2])
# CA_FRN_CRF
df2 = anonymise_float(df2, "CA_FRN_CRF", [0.1,1.2])
# TOTAL_CA_DECLARE_FRN
df2 = anonymise_float(df2, "TOTAL_CA_DECLARE_FRN", [0.1,1.2])
# TOTAL_ENCAISSE
df2 = anonymise_float(df2, "TOTAL_ENCAISSE", [0.1,1.2])
# MTT_A_REVERSER_DIFF
df2 = anonymise_float(df2, "MTT_A_REVERSER_DIFF", [0.1,1.2])
# MTT_A_REV_DIFF_HORS_SYN
df2 = anonymise_float(df2, "MTT_A_REV_DIFF_HORS_SYN", [0.1,1.2])
# MTT_A_REVERSER_BCP
df2 = anonymise_float(df2, "MTT_A_REVERSER_BCP", [0.1,1.2])
# ECART_MTT_A_REVERSER
df2 = anonymise_float(df2, "ECART_MTT_A_REVERSER", [0.1,1.2])
# MTT_PREVISION_AVEC_SYNERGIE
df2 = anonymise_float(df2, "MTT_PREVISION_AVEC_SYNERGIE", [0.1,1.2])
# MTT_PREVISION_HORS_SYNERGIE
df2 = anonymise_float(df2, "MTT_PREVISION_HORS_SYNERGIE", [0.1,1.2])
# Cdfop
# Norca
# Cient
# Cdprolog
# Lbetblba
df2, key_lbetblba = anonymise_cat(df2, "Lbetblba", key_lbetblba)
# Activite
df2,_ = anonymise_cat(df2, "Activite", "ACT")
# Code_Ent_BCP
# Libelle_entrepot
# SOURCES
# Carpr
df2 = anonymise_float(df2,"Carpr",[0.1,1.2])
# Mteca
df2 = anonymise_float(df2,"Mteca",[0.1,1.2])
# _Ca_Bn
df2 = anonymise_float(df2,"_Ca_Bn",[0.1,1.2])
# _Ristourne_Prolog
df2 = anonymise_float(df2,"_Ristourne_Prolog",[0.1,1.2])
# Ecart_CA
df2 = anonymise_float(df2,"Ecart_CA",[0.1,1.2])
# %CA
df2 = anonymise_float(df2,"%CA",[0.1,1.2])
# Ecart_Ristourne
df2 = anonymise_float(df2,"Ecart_Ristourne",[0.1,1.2])
# %Ristourne
df2 = anonymise_float(df2,"%Ristourne",[0.1,1.2])
# DATE

df2 = df2[columns_order]

In [15]:
df2.head()

,CENTRALE,CODE_SOCIETE,LIBELLE_SOCIETE,CODE_CLIENT_BCP,POLE_AFFILIANT,FOURNISSEUR,NOM_FOURNISSEUR,REGROUPEMENT,CA_DECLARE_FRN,CA_FRN_CRF,...,SOURCES,Carpr,Mteca,_Ca_Bn,_Ristourne_Prolog,Ecart_CA,%CA,Ecart_Ristourne,%Ristourne,DATE
0,CENT 1,144,LIB 1,209,POLE 1,195,FRN 93,1,"0,0","227020,42738047626",...,NR,"168802,9740431369","562,309152525394","212203,446160535","59365,672881148195","-232,05774250298643","-0,0066284821826500095","-552,349007490598","-0,002359776388255481",30/09/2022
1,CENT 1,144,LIB 1,209,POLE 1,195,FRN 93,13,"0,0","63577,208336402204",...,NR,"54588,950351034255","172,27163859140794","68643,76895232024","1440,770786839551","1362,7181189569596","0,13899127805404712","-2,707836459500642","-0,00047935394426897526",30/09/2022
2,CENT 1,144,LIB 1,209,POLE 1,301,FRN 60,2,"0,0","343585,6802304508",...,NR,"257473,5177036143","-728,9942031396098","326920,9526291996","100462,80937226079","541,1218938943896","0,010212757207953147","1433,2176720953914","0,0036836690891159135",30/09/2022
3,CENT 1,144,LIB 1,209,POLE 1,301,FRN 60,9,"0,0","406466,30602617486",...,NR,"302098,98381280305","-316,81796329773647","382480,3238253324","114018,61797589483","-21,548437765887876","-0,00034377514637026497","-55,10433462078871","-0,0001233951618260479",30/09/2022
4,CENT 1,144,LIB 1,209,POLE 1,592,FRN 241,1,"0,0","279216,6616242243",...,NR,"193681,71904045285","37,34853760032907","244723,3926581005","55352,18237585102","-2805,8139939392936","-0,06516291691337793","-5465,958054005234","-0,02346406337496547",30/09/2022


In [16]:
df2.sample(1).T

,14128
CENTRALE,CENT 2
CODE_SOCIETE,1386
LIBELLE_SOCIETE,LIB 2
CODE_CLIENT_BCP,20096
POLE_AFFILIANT,POLE 1
FOURNISSEUR,44203
NOM_FOURNISSEUR,FRN 288
REGROUPEMENT,50
CA_DECLARE_FRN,"0,0"
CA_FRN_CRF,"15408,97258650516"


In [17]:
# Save
path_output = Path(cwd,'DONE/Anonym_Ristournes_N_2605.csv')
df2.to_csv(path_output, sep = ";", encoding = 'utf-8', index = False)

#### Anonym_Transco_Entrepot.xlsx

In [18]:
df3 = pd.read_excel(Path(cwd,'TO_DO/Anonym_Transco_Entrepot.xlsx'), sheet_name = "Transco_Entrepot_Etab_Diff")

In [19]:
df3.head()

,Cdpafd,Cdprolog,Cdetbthales,Libelle Prolog,Cdclinat,Lbetblba,MZ - CP,commentaires?,Nom Prolog,Activite,SOURCES
0,ENT GR,1.0,FRY565,NaN,7338,AIRE S/LA LYS -EPI,62120.0,0,NaN,EPI,NaN
1,ENT GR,2.0,FRY519,CARPIQ-AB,209,CARPIQUET -EPI,14650.0,0,CARPIQ-AB,EPI,NaN
2,ENT GR,4.0,FRY573,NaN,369,MACON -EPI,71000.0,0,NaN,EPI,NaN
3,ENT GR,7.0,FRY8V8,NIMES C,7369,NIMES -EPI C,30000.0,0,NIMES C,EPI,NaN
4,ENT GR,9.0,FRY8U2,NaN,7382,VALLEIRY -EPI,74520.0,0,NaN,EPI,NaN


In [20]:
df3.sample(1).T

,340
Cdpafd,ENT GR
Cdprolog,NaN
Cdetbthales,NaN
Libelle Prolog,NaN
Cdclinat,19647
Lbetblba,FER ENT BEZIERS -FL FT
MZ - CP,34500.0
commentaires?,0
Nom Prolog,NaN
Activite,F & L


In [21]:
columns_order = list(df3.columns)

# Cdpafd
df3,_ = anonymise_cat(df3,"Cdpafd","POLE")
# Cdprolog
# Cdetbthales
# Libelle Prolog
df3,_ = anonymise_cat(df3,"Libelle Prolog","LBL")
# Cdclinat
# Lbetblba
df3,_ = anonymise_cat(df3,"Lbetblba", key_lbetblba)
# MZ - CP 
# commentaires?
# Nom Prolog
df3,_ = anonymise_cat(df3,"Nom Prolog","LBL")
# Activite
df3,_ = anonymise_cat(df3,"Activite","ACT")
# SOURCES

df3 = df3[columns_order]

In [22]:
df3.head()

,Cdpafd,Cdprolog,Cdetbthales,Libelle Prolog,Cdclinat,Lbetblba,MZ - CP,commentaires?,Nom Prolog,Activite,SOURCES
0,POLE 1,1.0,FRY565,NaN,7338,LBL 100,62120.0,0,NaN,ACT 1,NaN
1,POLE 1,2.0,FRY519,LBL 24,209,LBL 14,14650.0,0,LBL 24,ACT 1,NaN
2,POLE 1,4.0,FRY573,NaN,369,LBL 251,71000.0,0,NaN,ACT 1,NaN
3,POLE 1,7.0,FRY8V8,LBL 88,7369,LBL 250,30000.0,0,LBL 88,ACT 1,NaN
4,POLE 1,9.0,FRY8U2,NaN,7382,LBL 249,74520.0,0,NaN,ACT 1,NaN


In [23]:
df3.sample(1).T

,112
Cdpafd,POLE 1
Cdprolog,392.0
Cdetbthales,FRY8R2
Libelle Prolog,LBL 152
Cdclinat,9176
Lbetblba,LBL 27
MZ - CP,31830.0
commentaires?,0
Nom Prolog,LBL 152
Activite,ACT 5


In [24]:
# Save
path_output = Path(cwd,'DONE/Anonym_Transco_Entrepot_2605.xlsx')
with pd.ExcelWriter(
        path_output, mode="a", engine="openpyxl", if_sheet_exists="overlay"
    ) as writer:
        # Write data for global assignment (overlay mode)
        df3.to_excel(
            writer,
            sheet_name="Transco_Entrepot_Etab_Diff",
            index=False,
            header=False,
            startrow=1,
        )